In [ ]:
#This cell you will need to run it twice (First time will crash)
# !pip uninstall tensorflow -y && pip install torchtyping
!pip install typing-extensions --upgrade
!pip install pennylane update
!pip install tqdm
!pip install botorch

In [ ]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt
import time
import random
import torch
import torch.nn as nn
from tqdm import tqdm
from itertools import  product

from tqdm import trange  # <- Progress bar
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import ExpectedImprovement, LogExpectedImprovement, UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.utils.transforms import normalize, unnormalize
from gpytorch.mlls import ExactMarginalLogLikelihood

# Define the Quantum Circuit

In [ ]:
num_qubits = 2

# Define the Pauli operators
paulis = [qml.PauliX, qml.PauliY, qml.PauliZ]

# Generate all ordered pairs (X@X, X@Y, ..., Z@Z)
pauli_pairs = list(product(paulis, repeat=2))  # includes asymmetric combinations

list_of_observables = [
    p1(i) @ p2(i + 1)
    for i in range(num_qubits - 1)
    for p1, p2 in pauli_pairs
]
# # Generate all combinations of two Pauli operators (with replacement to allow X@X, Y@Y, etc.)
# pauli_pairs = list(combinations_with_replacement(paulis, 2))

print(list_of_observables)



In [ ]:
# draw https://docs.pennylane.ai/en/stable/code/api/pennylane.draw.html
dev = qml.device("lightning.qubit", wires=num_qubits)
@qml.qnode(qml.device('lightning.qubit', wires=num_qubits))
def circuit_test(params, **kwargs):
    observables = kwargs.pop("observable")
    for w in range(num_qubits):
        qml.Hadamard(wires=w)
        qml.RY(params[w], wires=w)
    for w in dev.wires[:-1]:
        qml.CNOT(wires=[w, w + 1])
    for w in dev.wires:
        qml.RZ(params[w + num_qubits], wires=w)
    return [qml.expval(o) for o in observables]

params = np.random.randn(2 * num_qubits)
for o in list_of_observables:
    print(qml.draw(circuit_test)(params, observable=[o]))

In [ ]:
def get_energy_function():
    dev = qml.device("lightning.qubit", wires=num_qubits, shots=10000000)
    dev_exact = qml.device("lightning.qubit", wires=num_qubits)

    def circuit_base(params, **kwargs):
      observables = kwargs.pop("observable")
      for w in range(num_qubits):
          qml.Hadamard(wires=w)
          qml.RY(params[w], wires=w)
      for w in dev.wires[:-1]:
          qml.CNOT(wires=[w, w + 1])
      for w in dev.wires:
          qml.RZ(params[w + num_qubits], wires=w)
      return [qml.expval(o) for o in observables]

    def energy_function(params):
        circuit_ex = qml.QNode(circuit_base, dev_exact)
        circuit = qml.QNode(circuit_base, dev)
        expval_exact = [
            circuit_ex(params, observable=[o]) for o in list_of_observables
        ]
        expval_measurements = [
            circuit(params, observable=[o]) for o in list_of_observables
        ]
        # return np.sum(np.array(expval_exact))
        return torch.tensor(np.sum(np.array(expval_exact)),dtype=torch.double)
    return energy_function

f_energy = get_energy_function()


# Bayesian Optimization Search

In [ ]:
def get_BO_trajectory(n_itr, acq_name='UCB'):
    # get initial values
    d = num_qubits * 2
    bounds = torch.stack([-torch.pi*torch.ones(d), torch.pi*torch.ones(d)]).to(torch.double)

    train_x = torch.rand((1, d)).double()
    train_y = f_energy(train_x[0]).reshape(1,1).double()
    print(train_x, train_y)

    dim = bounds.shape[1]
    bounds_normalized = torch.stack([torch.zeros(dim), torch.ones(dim)])
    for iteration in trange(n_itr, desc="BO iterations"):
        # Fit GP
        model = SingleTaskGP(train_x, train_y)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)

        # Acquisition function
        if acq_name == 'EI':
            acq_f = ExpectedImprovement(model=model, best_f=train_y.min(), maximize=False)
        elif acq_name == 'logEI':
            acq_f = LogExpectedImprovement(model=model, best_f=train_y.min(), maximize=False)
        elif acq_name == 'UCB':
            acq_f = UpperConfidenceBound(model=model, beta=0.1, maximize=False) # beta is a hyper parameter

        # Optimize acquisition function
        candidate, _ = optimize_acqf(
            acq_function=acq_f,
            bounds=bounds_normalized,
            q=1,
            num_restarts=5,
            raw_samples=20,
        )

        # Evaluate f_energy at new candidate
        new_x = unnormalize(candidate, bounds=bounds)
        new_y = f_energy(new_x[0].detach().numpy())

        # Update training data
        train_x = torch.cat([train_x, candidate])
        train_y = torch.cat([train_y, new_y.reshape(1,1)],0)

        # Optional: log progress
        tqdm.write(f"Iter {iteration + 1:2d}: y = {new_y.item():.4f} at x = {new_x.numpy()}")

    return train_x.detach().numpy(), train_y.detach().numpy()


In [ ]:
n_itr = 10
trj_x_ucb, trj_y_ucb = get_BO_trajectory(n_itr, acq_name='UCB')

In [ ]:
n_itr = 10
trj_x_ei, trj_y_ei = get_BO_trajectory(n_itr, acq_name='logEI')

# Inspect the Results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import torch

def plot_kde_comparison(train_y_all_1, train_y_all_2, labels=('Method A', 'Method B'), bandwidth=0.2):
    """
    Plot KDE of f_energy values from two different acquisition functions.

    Args:
        train_y_all_1: list of tensors (n_points, 1), for method 1
        train_y_all_2: list of tensors (n_points, 1), for method 2
        labels: tuple of two labels for the legend
        bandwidth: KDE smoothing parameter
    """
    # Flatten and convert to numpy
    y1 = train_y_all_1.flatten()
    y2 = train_y_all_2.flatten()

    # Colors: use fill+line match
    color1 = "#1b9e77"  # greenish teal
    color2 = "#d95f02"  # warm orange

    # Plot
    plt.figure(figsize=(7, 5))

    sns.kdeplot(x=y1, bw_adjust=bandwidth, label=labels[0], color=color1, fill=True, linewidth=2, alpha=0.6)
    sns.kdeplot(x=y2, bw_adjust=bandwidth, label=labels[1], color=color2, fill=True, linewidth=2, alpha=0.6)

    plt.xlabel("Energy")
    plt.ylabel("Density")
    plt.legend()

    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_kde_single(train_y_all_1, labels='Method A', bandwidth=0.2):
    """
    Plot KDE of f_energy values from two different acquisition functions.

    Args:
        train_y_all_1: list of tensors (n_points, 1), for method 1
        train_y_all_2: list of tensors (n_points, 1), for method 2
        labels: tuple of two labels for the legend
        bandwidth: KDE smoothing parameter
    """
    # Flatten and convert to numpy
    y1 = train_y_all_1.flatten()

    # Colors: use fill+line match
    color1 = "#1b9e77"  # greenish teal
    color2 = "#d95f02"  # warm orange

    # Plot
    plt.figure(figsize=(7, 5))

    sns.kdeplot(x=y1, bw_adjust=bandwidth, label=labels, color=color1, fill=True, linewidth=2, alpha=0.6)

    plt.xlabel("Energy")
    plt.ylabel("Density")
    plt.legend()

    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_kde_comparison(trj_y_ucb , trj_y_ei, labels=('UCB', 'logEI'), bandwidth=0.3)


# BO for H$_2$ minimal-basis UCCSD
The Unitary coupled cluster unitary operator within first-order Trotter approximation is: $\hat{U}(\vec{\theta}) =
\prod_{p > r} \mathrm{exp} \Big\{\theta_{pr}
(\hat{c}_p^\dagger \hat{c}_r-\mathrm{H.c.}) \Big\}
\prod_{p > q > r > s} \mathrm{exp} \Big\{\theta_{pqrs}
(\hat{c}_p^\dagger \hat{c}_q^\dagger \hat{c}_r \hat{c}_s-\mathrm{H.c.}) \Big\}$
which after employing the Jordan-WIgner transformation becomes:

\begin{split}\hat{U}(\vec{\theta}) = && \prod_{p > r} \mathrm{exp} \Big\{ \frac{i\theta_{pr}}{2}
\bigotimes_{a=r+1}^{p-1} \hat{Z}_a (\hat{Y}_r \hat{X}_p - \mathrm{H.c.}) \Big\} \\
&& \times \prod_{p > q > r > s} \mathrm{exp} \Big\{ \frac{i\theta_{pqrs}}{8}
\bigotimes_{b=s+1}^{r-1} \hat{Z}_b \bigotimes_{a=q+1}^{p-1}
\hat{Z}_a (\hat{X}_s \hat{X}_r \hat{Y}_q \hat{X}_p +
\hat{Y}_s \hat{X}_r \hat{Y}_q \hat{Y}_p +
\hat{X}_s \hat{Y}_r \hat{Y}_q \hat{Y}_p +
\hat{X}_s \hat{X}_r \hat{X}_q \hat{Y}_p -
\{\mathrm{H.c.}\}) \Big\}.\end{split}

The expression looks complicated. Luckily,  we can use Pennylane to directly implement it

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the molecule
symbols  = ['H', 'H']
geometry = np.array([[0.0,  0.0,  0.0],
                     [0.0,  0.0,  1.2]], requires_grad = False)
electrons = 2
charge = 0

# Build the electronic Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)

# Define the HF state
hf_state = qml.qchem.hf_state(electrons, qubits)

# Generate single and double excitations
singles, doubles = qml.qchem.excitations(electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

# Define the device
dev = qml.device("lightning.qubit", wires=qubits)

# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))

# Define the optimizer
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

# Optimize the circuit parameters and compute the energy
for n in range(40):
    params, energy = optimizer.step_and_cost(circuit, params,
    wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
print(params)


In [ ]:
(qml.draw(circuit)(params, wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state))

In [ ]:
print(qml.draw(circuit)(params, wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state))

Now let's see how this is done with Bayesian optimization.

In [ ]:
def get_energy_function():
    # Define the molecule
    symbols  = ['H', 'H']
    geometry = np.array([[0.0,  0.0,  0.0],
                     [0.0,  0.0,  1.2]], requires_grad = False)
    electrons = 2
    charge = 0

    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge, basis='sto-3g')

    # Define the HF state
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Generate single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)

    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    dev = qml.device("lightning.qubit", wires=qubits, shots=1)
    dev_exact = qml.device("lightning.qubit", wires=qubits)

    def circuit_base(params, wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state):
        qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
        return qml.expval(H)

    def energy_function(params):
        circuit_ex = qml.QNode(circuit_base, dev_exact)
        circuit = qml.QNode(circuit_base, dev)
        expval_exact = circuit_ex(params) #circuit_ex(params)

        return torch.tensor(expval_exact,dtype=torch.double)
        #return torch.tensor(np.sum(np.array(expval_exact)),dtype=torch.double)
    return energy_function

f_energy = get_energy_function()

In [ ]:
params = np.zeros(len(singles) + len(doubles))
f_energy(params)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

In [ ]:
def get_BO_trajectory(n_itr, acq_name='UCB'):
    # get initial values
    d = len(singles) + len(doubles)#num_qubits * 2
    bounds = torch.stack([-torch.pi*torch.ones(d), torch.pi*torch.ones(d)]).to(torch.double)

    train_x = torch.rand((1, d)).double()
    train_y = f_energy(train_x[0]).reshape(1,1).double()

    dim = bounds.shape[1]
    bounds_normalized = torch.stack([torch.zeros(dim), torch.ones(dim)])
    for iteration in trange(n_itr, desc="BO iterations"):
        # Fit GP
        model = SingleTaskGP(train_x, train_y,)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)

        # Acquisition function
        if acq_name == 'EI':
            acq_f = ExpectedImprovement(model=model, best_f=train_y.min(), maximize=False)
        elif acq_name == 'logEI':
            acq_f = LogExpectedImprovement(model=model, best_f=train_y.min(), maximize=False)
        elif acq_name == 'UCB':
            acq_f = UpperConfidenceBound(model=model, beta=0.25, maximize=False) # beta is a hyper parameter

        # Optimize acquisition function
        candidate, _ = optimize_acqf(
            acq_function=acq_f,
            bounds=bounds_normalized,
            q=1,
            num_restarts=5,
            raw_samples=20,
        )

        # Evaluate f_energy at new candidate
        new_x = unnormalize(candidate, bounds=bounds)
        new_y = f_energy(new_x[0].detach().numpy())

        # Update training data
        train_x = torch.cat([train_x, candidate])
        train_y = torch.cat([train_y, new_y.reshape(1,1)],0)

        # Optional: log progress
        tqdm.write(f"Iter {iteration + 1:2d}: y = {new_y.item():.4f} at x = {new_x.numpy()}")

    return train_x.detach().numpy(), train_y.detach().numpy()


In [ ]:
n_itr = 100
trj_x_ucb, trj_y_ucb = get_BO_trajectory(n_itr, acq_name='UCB')

In [ ]:
n_itr = 100
trj_x_ei, trj_y_ei = get_BO_trajectory(n_itr, acq_name='logEI')

In [ ]:
plot_kde_comparison(trj_y_ucb , trj_y_ei, labels=('UCB', 'logEI'), bandwidth=0.05)

In [ ]:
plot_kde_single(trj_y_ei, labels='logEI', bandwidth=0.05)

In [ ]:
print("Minimum energy UCB: {} Minimum energy EI: {}".format(min(trj_y_ucb.flatten()), min(trj_y_ei.flatten())))

We see that the results capture the correct energy :)

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from sklearn.decomposition import PCA


def animate_bo(train_x_all, train_y_all, bounds, f_energy, fixed_vals=(-np.pi, np.pi), acq_name = 'UCB', filename='bo_animation.mp4', cache_dir='/content/bo_cache'):
    """
    Animate Bayesian Optimization sampling process on a 3D function projected to 2D via PCA.
    Includes caching of the grid and PCA projection to speed up repeated calls.
    """
    from IPython.display import HTML
    from scipy.interpolate import griddata

    train_x_all = unnormalize(torch.tensor(train_x_all), bounds=bounds)
    X_train = train_x_all.detach().numpy()
    train_y_all = torch.tensor(train_y_all)

    # Create a grid
    lin = torch.linspace(fixed_vals[0], fixed_vals[1], 25)
    X1, X2 = torch.meshgrid(lin, lin, indexing='ij')
    X3 = torch.stack([X1.reshape(-1), X2.reshape(-1), torch.ones_like(X2.reshape(-1))], dim=-1)
    Xx3 = normalize(X3, bounds=bounds)
    pca = PCA(n_components=2)
    Z2 = pca.fit_transform(X3)
    Z2 = Z2.T
    Z2 = torch.tensor(Z2, dtype=torch.float)

    Y = []
    for xi in X3:
        Y.append(f_energy(xi).detach())
    Y3 = torch.tensor(Y)
    i0 = np.argmin(Y3)
    y_best = Y3[i0]
    x_best = X3[i0]
    Y3 = Y3.reshape(X1.shape)

    # Set up figure
    fig, axs = plt.subplots(1, 1, figsize=(10, 5))

    def animate(i):
        axs.cla()


        # Plot true function
        axs.contourf(lin,lin,Y3)
        axs.scatter(X_train[:i,0],X_train[:i,1], c='r', edgecolors='k')
        axs.scatter(x_best[0],x_best[1], c='k', edgecolors='k')
        axs.set_title(f'True Function, Step {i+1}, {acq_name}')

    ani = animation.FuncAnimation(fig, animate, frames=len(train_x_all), interval=500)
    plt.close()

    return HTML(ani.to_jshtml())


In [ ]:
d = len(singles) + len(doubles)
bounds = torch.stack([-torch.pi*torch.ones(d), torch.pi*torch.ones(d)]).to(torch.double)
animate_bo(trj_x_ucb, trj_y_ucb, bounds, f_energy, fixed_vals=(-np.pi, np.pi), acq_name ='UCB',filename='bo_animation.mp4', cache_dir='bo_cache')

In [ ]:
d = len(singles) + len(doubles)
bounds = torch.stack([-torch.pi*torch.ones(d), torch.pi*torch.ones(d)]).to(torch.double)
animate_bo(trj_x_ei, trj_y_ei, bounds, f_energy, fixed_vals=(-np.pi, np.pi), acq_name ='EI',filename='bo_animation.mp4', cache_dir='bo_cache')